In [12]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import vectorbtpro as vbt

from vctr.data.data_loader import get_data
import numpy as np
from scipy.stats import spearmanr
from vctr.data.labeling import label_data_extrema_multi


def overlap(arr):
    corr, _ = spearmanr(labels.values, arr)
    return corr


data = get_data('FTM', '15m')
labels = label_data_extrema_multi(data).label
labels.loc[labels == 2] = 1


/var/folders/zv/6wx6x5ps6s11ny3rshxvz26c0000gn/T/ipykernel_56644/3716074469.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels.loc[labels == 2] = 1


In [ ]:
def calculate_overlap_percent(series):
    if not labels.index.equals(series.index):
        raise ValueError('Series must have the same index')

    total_elements = len(labels)

    regular_overlap = (labels == series).sum()
    regular_overlap_percent = (regular_overlap / total_elements) * 100

    inverted_series = 1 - series
    inverted_overlap = (labels == inverted_series).sum()
    inverted_overlap_percent = (inverted_overlap / total_elements) * 100

    return max(regular_overlap_percent, inverted_overlap_percent)


In [ ]:
from vctr.features.feature_engineering import add_rsi_features


@vbt.parameterized(merge_func='concat', show_progress=True)
def opt_rsi(df, rsi_period, ema_period, th_low, th_high):
    df2 = df.copy()
    drop_cols = df2.columns
    df2 = add_rsi_features(df2, rsi_period, ema_period, th_low, th_high)
    df2 = df2.drop(drop_cols, axis=1)
    # Drop all columns that only contain a single value
    df2 = df2.loc[:, df2.nunique() != 1]

    results = [calculate_overlap_percent(df2[col]) for col in df2.columns]

    return max(results)


In [ ]:
rsi_periods = vbt.Param(list(range(2, 60, 2)))
ema_periods = vbt.Param(list(range(2, 60, 2)))
th_low = vbt.Param(list(range(10, 40, 10)))
th_high = vbt.Param(list(range(60, 90, 10)))

res = opt_rsi(data, rsi_periods, ema_periods, th_low, th_high)


In [ ]:
pd.set_option('display.max_rows', None)
res.sort_values(ascending=False).head(100)

In [17]:
from PAMI.frequentPattern.basic import FPGrowth as alg

# df = pd.DataFrame(labels)
# df.to_csv('mining.csv')
# label


obj = alg.FPGrowth('mining.csv',100,'\t')
obj.startMine()


# obj.savePatterns('frequentPatters_100.txt')
# df = obj.getPatternsAsDataFrame()


# print('Runtime: ' + str(obj.getRuntime()))
# print('Memory: ' + str(obj.getMemoryRSS()))

Frequent patterns were generated successfully using frequentPatternGrowth algorithm


In [18]:
df2 = obj.getPatternsAsDataFrame()

In [19]:
df2

{}